# Loading everything

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from statistics import mean

In [2]:
batch1 = pickle.load(open(r'batch1.pkl', 'rb'))
#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

In [3]:
numBat1 = len(batch1.keys())
numBat1

41

In [4]:
batch2 = pickle.load(open(r'batch2.pkl','rb'))

In [5]:
# There are four cells from batch1 that carried into batch2, we'll remove the data from batch2
# and put it with the correct cell from batch1
batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16']
batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
add_len = [662, 981, 1060, 208, 482];

In [6]:
for i, bk in enumerate(batch1_keys):
    batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
    for j in batch1[bk]['summary'].keys():
        if j == 'cycle':
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
        else:
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
    last_cycle = len(batch1[bk]['cycles'].keys())
    for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
        batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]

In [7]:
del batch2['b2c7']
del batch2['b2c8']
del batch2['b2c9']
del batch2['b2c15']
del batch2['b2c16']

In [8]:
numBat2 = len(batch2.keys())
numBat2

43

In [9]:
batch3 = pickle.load(open(r'batch3.pkl','rb'))
# remove noisy channels from batch3
del batch3['b3c37']
del batch3['b3c2']
del batch3['b3c23']
del batch3['b3c32']
del batch3['b3c42']
del batch3['b3c43']

In [10]:
numBat3 = len(batch3.keys())
numBat3

40

In [11]:
numBat = numBat1 + numBat2 + numBat3
numBat

124

In [12]:
bat_dict = {**batch1, **batch2, **batch3}

In [13]:
for i in bat_dict.keys():
    print(i)

b1c0
b1c1
b1c2
b1c3
b1c4
b1c5
b1c6
b1c7
b1c9
b1c11
b1c14
b1c15
b1c16
b1c17
b1c18
b1c19
b1c20
b1c21
b1c23
b1c24
b1c25
b1c26
b1c27
b1c28
b1c29
b1c30
b1c31
b1c32
b1c33
b1c34
b1c35
b1c36
b1c37
b1c38
b1c39
b1c40
b1c41
b1c42
b1c43
b1c44
b1c45
b2c0
b2c1
b2c2
b2c3
b2c4
b2c5
b2c6
b2c10
b2c11
b2c12
b2c13
b2c14
b2c17
b2c18
b2c19
b2c20
b2c21
b2c22
b2c23
b2c24
b2c25
b2c26
b2c27
b2c28
b2c29
b2c30
b2c31
b2c32
b2c33
b2c34
b2c35
b2c36
b2c37
b2c38
b2c39
b2c40
b2c41
b2c42
b2c43
b2c44
b2c45
b2c46
b2c47
b3c0
b3c1
b3c3
b3c4
b3c5
b3c6
b3c7
b3c8
b3c9
b3c10
b3c11
b3c12
b3c13
b3c14
b3c15
b3c16
b3c17
b3c18
b3c19
b3c20
b3c21
b3c22
b3c24
b3c25
b3c26
b3c27
b3c28
b3c29
b3c30
b3c31
b3c33
b3c34
b3c35
b3c36
b3c38
b3c39
b3c40
b3c41
b3c44
b3c45


# Getting all the features

In [14]:
from scipy.stats import skew, kurtosis
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

In [15]:
features_df = pd.DataFrame()

In [16]:
minimum_dQ_100_10 = np.zeros(len(bat_dict.keys()))
variance_dQ_100_10 = np.zeros(len(bat_dict.keys()))
skewness_dQ_100_10 = np.zeros(len(bat_dict.keys()))
kurtosis_dQ_100_10 = np.zeros(len(bat_dict.keys()))

for i, cell in enumerate(bat_dict.values()):
    c10 = cell['cycles']['10']
    c100 = cell['cycles']['100']
    dQ_100_10 = c100['Qdlin'] - c10['Qdlin']
    
    minimum_dQ_100_10[i] = np.log(np.abs(np.min(dQ_100_10)))
    variance_dQ_100_10[i] = np.log(np.var(dQ_100_10))
    skewness_dQ_100_10[i] = np.log(np.abs(skew(dQ_100_10)))
    kurtosis_dQ_100_10[i] = np.log(np.abs(kurtosis(dQ_100_10)))

features_df["minimum_dQ_100_10"] = minimum_dQ_100_10
features_df["variance_dQ_100_10"] = variance_dQ_100_10
features_df["skewness_dQ_100_10"] = skewness_dQ_100_10
features_df["kurtosis_dQ_100_10"] = kurtosis_dQ_100_10


slope_lin_fit_2_100 = np.zeros(len(bat_dict.keys()))
intercept_lin_fit_2_100 = np.zeros(len(bat_dict.keys()))
discharge_capacity_2 = np.zeros(len(bat_dict.keys()))
diff_discharge_capacity_max_2 = np.zeros(len(bat_dict.keys()))

for i, cell in enumerate(bat_dict.values()):
    # Compute linear fit for cycles 2 to 100:
    q = cell['summary']['QD'][1:100].reshape(-1, 1)  # discharge cappacities; q.shape = (99, 1); 
    X = cycle_numbers = cell['summary']['cycle'][1:100].reshape(-1, 1)  # Cylce index from 2 to 100; X.shape = (99, 1)
    
    linear_regressor_2_100 = LinearRegression()
    linear_regressor_2_100.fit(X, q)
    
    slope_lin_fit_2_100[i] = linear_regressor_2_100.coef_[0]
    intercept_lin_fit_2_100[i] = linear_regressor_2_100.intercept_
    discharge_capacity_2[i] = q[0][0]
    diff_discharge_capacity_max_2[i] = np.max(q) - q[0][0]
    
features_df["slope_lin_fit_2_100"] = slope_lin_fit_2_100
features_df["intercept_lin_fit_2_100"] = intercept_lin_fit_2_100
features_df["discharge_capacity_2"] = discharge_capacity_2
features_df["diff_discharge_capacity_max_2"] = diff_discharge_capacity_max_2

mean_charge_time = np.zeros(len(bat_dict.keys()))
minimum_IR = np.zeros(len(bat_dict.keys()))
diff_IR_100_2 = np.zeros(len(bat_dict.keys()))


for i, cell in enumerate(bat_dict.values()):
    mean_charge_time[i] = np.mean(cell['summary']['chargetime'][1:6])
    minimum_IR[i] = np.min(cell['summary']['IR'][1:100])
    diff_IR_100_2[i] = cell['summary']['IR'][100] - cell['summary']['IR'][1]
    
features_df["mean_charge_time"] = mean_charge_time
features_df["minimum_IR"] = minimum_IR
features_df["diff_IR_100_2"] = diff_IR_100_2

In [17]:
features_df.head()

,minimum_dQ_100_10,variance_dQ_100_10,skewness_dQ_100_10,kurtosis_dQ_100_10,slope_lin_fit_2_100,intercept_lin_fit_2_100,discharge_capacity_2,diff_discharge_capacity_max_2,mean_charge_time,minimum_IR,diff_IR_100_2
0,-4.720901,-11.631570,-0.319238,-0.094314,-0.000207,1.091144,1.070689,0.468365,13.374894,0.016444,-0.000092
1,-4.709132,-11.825562,-1.148991,0.037823,0.000006,1.080965,1.075301,0.009288,13.409150,0.000000,-0.000034
2,-4.575223,-11.402131,-0.826717,0.160194,0.000010,1.084846,1.079922,0.008131,13.358242,0.000000,-0.000009
3,-3.922042,-10.099939,-0.741646,0.117089,0.000017,1.084075,1.079723,0.006530,12.025140,0.000000,0.000044
4,-4.230762,-10.602896,-1.052154,0.308340,0.000019,1.081875,1.078417,0.005900,12.041851,0.000000,-0.000036


In [18]:
features_df.shape

(124, 11)

In [19]:
cols = features_df.columns

# Splitting the data 80-20

In [20]:
target_df = pd.DataFrame()
cycle_life = []
for i in bat_dict:
    cycle_life.append(np.float(bat_dict[i]["cycle_life"]))
cycle_life = pd.Series(cycle_life)
target_df["cycle_life"] = cycle_life
target_df.head()

,cycle_life
0,1852.0
1,2160.0
2,2237.0
3,1434.0
4,1709.0


In [21]:
x = features_df
y = cycle_life

In [22]:
def find_outliers_IQR(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
   return outliers

In [23]:
find_outliers_IQR(x["variance_dQ_100_10"])

0    -11.631570
1    -11.825562
2    -11.402131
42    -6.296653
Name: variance_dQ_100_10, dtype: float64

In [24]:
x.drop([0,1,2,42], axis=0, inplace=True)
y.drop([0,1,2,42], axis=0, inplace=True)

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

In [26]:
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# APPROACH 1

## VARIANCE

In [27]:
error_list = []
real_lifetime_list = []
predicted_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the most similar cell
    minimum = min(df["distance"])
    
    #calculating the error
    real_lifetime = y_test[i]
    real_lifetime_list.append(float(real_lifetime))

    predicted_lifetime = y_train[df.loc[df['distance'] == minimum].index]
    predicted_lifetime_list.append(float(predicted_lifetime))

    error = float(abs(real_lifetime - predicted_lifetime))
    error = error/real_lifetime
    error_list.append(error*100)

    #print
    print(df)
    print("Error: ", np.array(error))

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipyk

    cell_test  cell_train  distance
0   -10.21142  -10.099939  0.111482
1   -10.21142  -10.602896  0.391476
2   -10.21142   -9.569157  0.642263
3   -10.21142   -8.683700  1.527721
4   -10.21142   -8.800099  1.411322
..        ...         ...       ...
91  -10.21142   -9.071859  1.139561
92  -10.21142   -9.417628  0.793792
93  -10.21142   -9.245718  0.965702
94  -10.21142   -9.190990  1.020430
95  -10.21142  -10.787515  0.576095

[96 rows x 3 columns]
Error:  0.18022813688212927
    cell_test  cell_train  distance
0   -9.754075  -10.099939  0.345864
1   -9.754075  -10.602896  0.848821
2   -9.754075   -9.569157  0.184918
3   -9.754075   -8.683700  1.070375
4   -9.754075   -8.800099  0.953976
..        ...         ...       ...
91  -9.754075   -9.071859  0.682216
92  -9.754075   -9.417628  0.336446
93  -9.754075   -9.245718  0.508357
94  -9.754075   -9.190990  0.563085
95  -9.754075  -10.787515  1.033441

[96 rows x 3 columns]
Error:  0.11954624781849912
    cell_test  cell_train  distanc

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipyk

In [28]:
pd.Series(error_list).describe()

count    24.000000
mean     14.749057
std      10.991689
min       1.845018
25%       6.946591
50%      12.065137
75%      20.323915
max      35.151515
dtype: float64

## DISCHARGE

In [29]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the most similar cell
    minimum = min(df["distance"])
    
    #calculating the error
    real_lifetime = y_test[i]
    real_lifetime_list.append(float(real_lifetime))

    predicted_lifetime = y_train[df.loc[df['distance'] == minimum].index]
    predicted_lifetime_list.append(float(predicted_lifetime))

    error = float(abs(real_lifetime - predicted_lifetime))
    error = error/real_lifetime
    error_list.append(error*100)

    #print
    print(df)
    print("Error: ", np.array(error))

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -10.21142           -10.099939          -3.995719   
1            -10.21142           -10.602896          -3.995719   
2            -10.21142            -9.569157          -3.995719   
3            -10.21142            -8.683700          -3.995719   
4            -10.21142            -8.800099          -3.995719   
..                 ...                  ...                ...   
91           -10.21142            -9.071859          -3.995719   
92           -10.21142            -9.417628          -3.995719   
93           -10.21142            -9.245718          -3.995719   
94           -10.21142            -9.190990          -3.995719   
95           -10.21142           -10.787515          -3.995719   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -3.319977            -0.741646   
1            -4.230762           -3.319977            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.692178           -10.099939          -3.706722   
1            -9.692178           -10.602896          -3.706722   
2            -9.692178            -9.569157          -3.706722   
3            -9.692178            -8.683700          -3.706722   
4            -9.692178            -8.800099          -3.706722   
..                 ...                  ...                ...   
91           -9.692178            -9.071859          -3.706722   
92           -9.692178            -9.417628          -3.706722   
93           -9.692178            -9.245718          -3.706722   
94           -9.692178            -9.190990          -3.706722   
95           -9.692178           -10.787515          -3.706722   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -1.167855            -0.741646   
1            -4.230762           -1.167855            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -8.80387           -10.099939          -3.317319   
1             -8.80387           -10.602896          -3.317319   
2             -8.80387            -9.569157          -3.317319   
3             -8.80387            -8.683700          -3.317319   
4             -8.80387            -8.800099          -3.317319   
..                 ...                  ...                ...   
91            -8.80387            -9.071859          -3.317319   
92            -8.80387            -9.417628          -3.317319   
93            -8.80387            -9.245718          -3.317319   
94            -8.80387            -9.190990          -3.317319   
95            -8.80387           -10.787515          -3.317319   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -2.330745            -0.741646   
1            -4.230762           -2.330745            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.230704           -10.099939          -4.031907   
1           -10.230704           -10.602896          -4.031907   
2           -10.230704            -9.569157          -4.031907   
3           -10.230704            -8.683700          -4.031907   
4           -10.230704            -8.800099          -4.031907   
..                 ...                  ...                ...   
91          -10.230704            -9.071859          -4.031907   
92          -10.230704            -9.417628          -4.031907   
93          -10.230704            -9.245718          -4.031907   
94          -10.230704            -9.190990          -4.031907   
95          -10.230704           -10.787515          -4.031907   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -1.208466            -0.741646   
1            -4.230762           -1.208466            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [30]:
pd.Series(error_list).describe()

count    48.000000
mean     17.026748
std      13.062876
min       1.845018
25%       8.074167
50%      13.221385
75%      24.168636
max      58.823529
dtype: float64

## FULL MODEL

In [31]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the most similar cell
    minimum = min(df["distance"])
    
    #calculating the error
    real_lifetime = y_test[i]
    real_lifetime_list.append(float(real_lifetime))

    predicted_lifetime = y_train[df.loc[df['distance'] == minimum].index]
    predicted_lifetime_list.append(float(predicted_lifetime))

    error = float(abs(real_lifetime - predicted_lifetime))
    error = error/real_lifetime
    error_list.append(error*100)

    #print
    print(df)
    print("Error: ", np.array(error))


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -10.21142           -10.099939          -3.995719   
1            -10.21142           -10.602896          -3.995719   
2            -10.21142            -9.569157          -3.995719   
3            -10.21142            -8.683700          -3.995719   
4            -10.21142            -8.800099          -3.995719   
..                 ...                  ...                ...   
91           -10.21142            -9.071859          -3.995719   
92           -10.21142            -9.417628          -3.995719   
93           -10.21142            -9.245718          -3.995719   
94           -10.21142            -9.190990          -3.995719   
95           -10.21142           -10.787515          -3.995719   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000006   
1            -4.230762                      -0.000006   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Error:  0.12740141557128412
    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.692178           -10.099939          -3.706722   
1            -9.692178           -10.602896          -3.706722   
2            -9.692178            -9.569157          -3.706722   
3            -9.692178            -8.683700          -3.706722   
4            -9.692178            -8.800099          -3.706722   
..                 ...                  ...                ...   
91           -9.692178            -9.071859          -3.706722   
92           -9.692178            -9.417628          -3.706722   
93           -9.692178            -9.245718          -3.706722   
94           -9.692178            -9.190990          -3.706722   
95           -9.692178           -10.787515          -3.706722   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                  -4.288423e-07   
1            -4.230762                  -4.288423e-07   
2      

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Error:  0.1721183800623053
    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.88592           -10.099939          -3.826262   
1             -9.88592           -10.602896          -3.826262   
2             -9.88592            -9.569157          -3.826262   
3             -9.88592            -8.683700          -3.826262   
4             -9.88592            -8.800099          -3.826262   
..                 ...                  ...                ...   
91            -9.88592            -9.071859          -3.826262   
92            -9.88592            -9.417628          -3.826262   
93            -9.88592            -9.245718          -3.826262   
94            -9.88592            -9.190990          -3.826262   
95            -9.88592           -10.787515          -3.826262   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000021   
1            -4.230762                      -0.000021   
2       

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning


Error:  0.33542713567839194
    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.66936           -10.099939          -3.704429   
1             -9.66936           -10.602896          -3.704429   
2             -9.66936            -9.569157          -3.704429   
3             -9.66936            -8.683700          -3.704429   
4             -9.66936            -8.800099          -3.704429   
..                 ...                  ...                ...   
91            -9.66936            -9.071859          -3.704429   
92            -9.66936            -9.417628          -3.704429   
93            -9.66936            -9.245718          -3.704429   
94            -9.66936            -9.190990          -3.704429   
95            -9.66936           -10.787515          -3.704429   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                        0.00001   
1            -4.230762                        0.00001   
2      

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [32]:
pd.Series(error_list).describe()

count    72.000000
mean     16.686884
std      13.076921
min       0.000000
25%       7.685270
50%      13.221385
75%      21.677274
max      58.823529
dtype: float64

# APPROACH 2 (3 most similar cells)

## VARIANCE

In [33]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the 3 most similar cells
    three_most_similar_cells = df.nsmallest(3, 'distance')
    number1 = int(y_train[three_most_similar_cells.iloc[0].name])
    number2 = int(y_train[three_most_similar_cells.iloc[1].name])
    number3 = int(y_train[three_most_similar_cells.iloc[2].name])

    #getting max and min
    list_of_three_most_similar_cells = [number1, number2, number3]
    max_predicted_lifetime = max(list_of_three_most_similar_cells)
    min_predicted_lifetime = min(list_of_three_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))

    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")
    
    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

    cell_test  cell_train  distance
0   -10.21142  -10.099939  0.111482
1   -10.21142  -10.602896  0.391476
2   -10.21142   -9.569157  0.642263
3   -10.21142   -8.683700  1.527721
4   -10.21142   -8.800099  1.411322
..        ...         ...       ...
91  -10.21142   -9.071859  1.139561
92  -10.21142   -9.417628  0.793792
93  -10.21142   -9.245718  0.965702
94  -10.21142   -9.190990  1.020430
95  -10.21142  -10.787515  0.576095

[96 rows x 3 columns]
Predicted lifetime range is from:  1078 to  1836
Actual lifetime:  1315


    cell_test  cell_train  distance
0   -9.754075  -10.099939  0.345864
1   -9.754075  -10.602896  0.848821
2   -9.754075   -9.569157  0.184918
3   -9.754075   -8.683700  1.070375
4   -9.754075   -8.800099  0.953976
..        ...         ...       ...
91  -9.754075   -9.071859  0.682216
92  -9.754075   -9.417628  0.336446
93  -9.754075   -9.245718  0.508357
94  -9.754075   -9.190990  0.563085
95  -9.754075  -10.787515  1.033441

[96 rows x 3 columns]
Predicted lifeti

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

In [34]:
print(((accuracy_list.count("No"))/25)*100)
print(mean(range_list))

44.0
282.1666666666667


## DISCHARGE

In [35]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the 3 most similar cells
    three_most_similar_cells = df.nsmallest(3, 'distance')
    number1 = int(y_train[three_most_similar_cells.iloc[0].name])
    number2 = int(y_train[three_most_similar_cells.iloc[1].name])
    number3 = int(y_train[three_most_similar_cells.iloc[2].name])

    #getting max and min
    list_of_three_most_similar_cells = [number1, number2, number3]
    max_predicted_lifetime = max(list_of_three_most_similar_cells)
    min_predicted_lifetime = min(list_of_three_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

    
        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -10.21142           -10.099939          -3.995719   
1            -10.21142           -10.602896          -3.995719   
2            -10.21142            -9.569157          -3.995719   
3            -10.21142            -8.683700          -3.995719   
4            -10.21142            -8.800099          -3.995719   
..                 ...                  ...                ...   
91           -10.21142            -9.071859          -3.995719   
92           -10.21142            -9.417628          -3.995719   
93           -10.21142            -9.245718          -3.995719   
94           -10.21142            -9.190990          -3.995719   
95           -10.21142           -10.787515          -3.995719   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -3.319977            -0.741646   
1            -4.230762           -3.319977            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.432141           -10.099939          -3.598892   
1            -9.432141           -10.602896          -3.598892   
2            -9.432141            -9.569157          -3.598892   
3            -9.432141            -8.683700          -3.598892   
4            -9.432141            -8.800099          -3.598892   
..                 ...                  ...                ...   
91           -9.432141            -9.071859          -3.598892   
92           -9.432141            -9.417628          -3.598892   
93           -9.432141            -9.245718          -3.598892   
94           -9.432141            -9.190990          -3.598892   
95           -9.432141           -10.787515          -3.598892   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -1.167912            -0.741646   
1            -4.230762           -1.167912            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

...

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be


Predicted lifetime range is from:  713 to  880
Actual lifetime:  858


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.540689           -10.099939          -3.651003   
1            -9.540689           -10.602896          -3.651003   
2            -9.540689            -9.569157          -3.651003   
3            -9.540689            -8.683700          -3.651003   
4            -9.540689            -8.800099          -3.651003   
..                 ...                  ...                ...   
91           -9.540689            -9.071859          -3.651003   
92           -9.540689            -9.417628          -3.651003   
93           -9.540689            -9.245718          -3.651003   
94           -9.540689            -9.190990          -3.651003   
95           -9.540689           -10.787515          -3.651003   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -1.117463            -0.741646   
1

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.230704           -10.099939          -4.031907   
1           -10.230704           -10.602896          -4.031907   
2           -10.230704            -9.569157          -4.031907   
3           -10.230704            -8.683700          -4.031907   
4           -10.230704            -8.800099          -4.031907   
..                 ...                  ...                ...   
91          -10.230704            -9.071859          -4.031907   
92          -10.230704            -9.417628          -4.031907   
93          -10.230704            -9.245718          -4.031907   
94          -10.230704            -9.190990          -4.031907   
95          -10.230704           -10.787515          -4.031907   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -1.208466            -0.741646   
1            -4.230762           -1.208466            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipyker

In [36]:
print(((accuracy_list.count("No"))/25)*100)
print(mean(range_list))

32.0
338.75


## FULL MODEL

In [37]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the 3 most similar cells
    three_most_similar_cells = df.nsmallest(3, 'distance')
    number1 = int(y_train[three_most_similar_cells.iloc[0].name])
    number2 = int(y_train[three_most_similar_cells.iloc[1].name])
    number3 = int(y_train[three_most_similar_cells.iloc[2].name])

    #getting max and min
    list_of_three_most_similar_cells = [number1, number2, number3]
    max_predicted_lifetime = max(list_of_three_most_similar_cells)
    min_predicted_lifetime = min(list_of_three_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -10.21142           -10.099939          -3.995719   
1            -10.21142           -10.602896          -3.995719   
2            -10.21142            -9.569157          -3.995719   
3            -10.21142            -8.683700          -3.995719   
4            -10.21142            -8.800099          -3.995719   
..                 ...                  ...                ...   
91           -10.21142            -9.071859          -3.995719   
92           -10.21142            -9.417628          -3.995719   
93           -10.21142            -9.245718          -3.995719   
94           -10.21142            -9.190990          -3.995719   
95           -10.21142           -10.787515          -3.995719   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000006   
1            -4.230762                      -0.000006   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Predicted lifetime range is from:  828 to  1115
Actual lifetime:  1155


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.432141           -10.099939          -3.598892   
1            -9.432141           -10.602896          -3.598892   
2            -9.432141            -9.569157          -3.598892   
3            -9.432141            -8.683700          -3.598892   
4            -9.432141            -8.800099          -3.598892   
..                 ...                  ...                ...   
91           -9.432141            -9.071859          -3.598892   
92           -9.432141            -9.417628          -3.598892   
93           -9.432141            -9.245718          -3.598892   
94           -9.432141            -9.190990          -3.598892   
95           -9.432141           -10.787515          -3.598892   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000007   
1            -4.230

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.465497           -10.099939          -3.602002   
1            -9.465497           -10.602896          -3.602002   
2            -9.465497            -9.569157          -3.602002   
3            -9.465497            -8.683700          -3.602002   
4            -9.465497            -8.800099          -3.602002   
..                 ...                  ...                ...   
91           -9.465497            -9.071859          -3.602002   
92           -9.465497            -9.417628          -3.602002   
93           -9.465497            -9.245718          -3.602002   
94           -9.465497            -9.190990          -3.602002   
95           -9.465497           -10.787515          -3.602002   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000004   
1            -4.230762                      -0.000004   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.71285           -10.099939          -3.749464   
1             -9.71285           -10.602896          -3.749464   
2             -9.71285            -9.569157          -3.749464   
3             -9.71285            -8.683700          -3.749464   
4             -9.71285            -8.800099          -3.749464   
..                 ...                  ...                ...   
91            -9.71285            -9.071859          -3.749464   
92            -9.71285            -9.417628          -3.749464   
93            -9.71285            -9.245718          -3.749464   
94            -9.71285            -9.190990          -3.749464   
95            -9.71285           -10.787515          -3.749464   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                       -0.00002   
1            -4.230762                       -0.00002   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [38]:
print(((accuracy_list.count("No"))/25)*100)
print(mean(range_list))

64.0
160.08333333333334


# APPROACH 3 (5 most similar cells)

## VARIANCE

In [39]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the most similar cells
    most_similar_cells = df.nsmallest(5, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")
    
    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

#creating CSV df
csv_df = pd.DataFrame()
csv_df["cell_number_test"] = pd.Series(list(x_test.index))
csv_df["max prediction"] = pd.Series(max_predicted_lifetime_list)
csv_df["min prediction"] = pd.Series(min_predicted_lifetime_list)
csv_df["real"] = pd.Series(real_lifetime_list)
csv_df["accuracy"] = pd.Series(accuracy_list)   
csv_df.to_csv('variance_app3a.csv') 

    cell_test  cell_train  distance
0   -10.21142  -10.099939  0.111482
1   -10.21142  -10.602896  0.391476
2   -10.21142   -9.569157  0.642263
3   -10.21142   -8.683700  1.527721
4   -10.21142   -8.800099  1.411322
..        ...         ...       ...
91  -10.21142   -9.071859  1.139561
92  -10.21142   -9.417628  0.793792
93  -10.21142   -9.245718  0.965702
94  -10.21142   -9.190990  1.020430
95  -10.21142  -10.787515  0.576095

[96 rows x 3 columns]
Predicted lifetime range is from:  1063 to  1836
Actual lifetime:  1315


    cell_test  cell_train  distance
0   -9.754075  -10.099939  0.345864
1   -9.754075  -10.602896  0.848821
2   -9.754075   -9.569157  0.184918
3   -9.754075   -8.683700  1.070375
4   -9.754075   -8.800099  0.953976
..        ...         ...       ...
91  -9.754075   -9.071859  0.682216
92  -9.754075   -9.417628  0.336446
93  -9.754075   -9.245718  0.508357
94  -9.754075   -9.190990  0.563085
95  -9.754075  -10.787515  1.033441

[96 rows x 3 columns]
Predicted lifeti

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

In [40]:
print(((accuracy_list.count("No"))/25)*100)
print(mean(range_list))

32.0
349.5


## DISCHARGE

In [41]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(5, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

#creating CSV df
csv_df = pd.DataFrame()
csv_df["cell_number_test"] = pd.Series(list(x_test.index))
csv_df["max prediction"] = pd.Series(max_predicted_lifetime_list)
csv_df["min prediction"] = pd.Series(min_predicted_lifetime_list)
csv_df["real"] = pd.Series(real_lifetime_list)
csv_df["accuracy"] = pd.Series(accuracy_list)   
csv_df.to_csv('discharge_app3a.csv') 

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -10.21142           -10.099939          -3.995719   
1            -10.21142           -10.602896          -3.995719   
2            -10.21142            -9.569157          -3.995719   
3            -10.21142            -8.683700          -3.995719   
4            -10.21142            -8.800099          -3.995719   
..                 ...                  ...                ...   
91           -10.21142            -9.071859          -3.995719   
92           -10.21142            -9.417628          -3.995719   
93           -10.21142            -9.245718          -3.995719   
94           -10.21142            -9.190990          -3.995719   
95           -10.21142           -10.787515          -3.995719   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -3.319977            -0.741646   
1            -4.230762           -3.319977            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.560392           -10.099939          -3.656142   
1            -9.560392           -10.602896          -3.656142   
2            -9.560392            -9.569157          -3.656142   
3            -9.560392            -8.683700          -3.656142   
4            -9.560392            -8.800099          -3.656142   
..                 ...                  ...                ...   
91           -9.560392            -9.071859          -3.656142   
92           -9.560392            -9.417628          -3.656142   
93           -9.560392            -9.245718          -3.656142   
94           -9.560392            -9.190990          -3.656142   
95           -9.560392           -10.787515          -3.656142   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -2.124784            -0.741646   
1            -4.230762           -2.124784            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -7.923983           -10.099939          -2.842207   
1            -7.923983           -10.602896          -2.842207   
2            -7.923983            -9.569157          -2.842207   
3            -7.923983            -8.683700          -2.842207   
4            -7.923983            -8.800099          -2.842207   
..                 ...                  ...                ...   
91           -7.923983            -9.071859          -2.842207   
92           -7.923983            -9.417628          -2.842207   
93           -7.923983            -9.245718          -2.842207   
94           -7.923983            -9.190990          -2.842207   
95           -7.923983           -10.787515          -2.842207   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -1.260004            -0.741646   
1            -4.230762           -1.260004            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.71285           -10.099939          -3.749464   
1             -9.71285           -10.602896          -3.749464   
2             -9.71285            -9.569157          -3.749464   
3             -9.71285            -8.683700          -3.749464   
4             -9.71285            -8.800099          -3.749464   
..                 ...                  ...                ...   
91            -9.71285            -9.071859          -3.749464   
92            -9.71285            -9.417628          -3.749464   
93            -9.71285            -9.245718          -3.749464   
94            -9.71285            -9.190990          -3.749464   
95            -9.71285           -10.787515          -3.749464   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -1.217557            -0.741646   
1            -4.230762           -1.217557            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [42]:
print(((accuracy_list.count("No"))/25)*100)
print(mean(range_list))

32.0
374.5416666666667


## FULL MODEL

In [43]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(5, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

#creating CSV df
csv_df = pd.DataFrame()
csv_df["cell_number_test"] = pd.Series(list(x_test.index))
csv_df["max prediction"] = pd.Series(max_predicted_lifetime_list)
csv_df["min prediction"] = pd.Series(min_predicted_lifetime_list)
csv_df["real"] = pd.Series(real_lifetime_list)
csv_df["accuracy"] = pd.Series(accuracy_list)   
csv_df.to_csv('fullmodel_app3a.csv') 

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -10.21142           -10.099939          -3.995719   
1            -10.21142           -10.602896          -3.995719   
2            -10.21142            -9.569157          -3.995719   
3            -10.21142            -8.683700          -3.995719   
4            -10.21142            -8.800099          -3.995719   
..                 ...                  ...                ...   
91           -10.21142            -9.071859          -3.995719   
92           -10.21142            -9.417628          -3.995719   
93           -10.21142            -9.245718          -3.995719   
94           -10.21142            -9.190990          -3.995719   
95           -10.21142           -10.787515          -3.995719   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000006   
1            -4.230762                      -0.000006   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Predicted lifetime range is from:  828 to  1115
Actual lifetime:  1155


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.432141           -10.099939          -3.598892   
1            -9.432141           -10.602896          -3.598892   
2            -9.432141            -9.569157          -3.598892   
3            -9.432141            -8.683700          -3.598892   
4            -9.432141            -8.800099          -3.598892   
..                 ...                  ...                ...   
91           -9.432141            -9.071859          -3.598892   
92           -9.432141            -9.417628          -3.598892   
93           -9.432141            -9.245718          -3.598892   
94           -9.432141            -9.190990          -3.598892   
95           -9.432141           -10.787515          -3.598892   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000007   
1            -4.230

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.692178           -10.099939          -3.706722   
1            -9.692178           -10.602896          -3.706722   
2            -9.692178            -9.569157          -3.706722   
3            -9.692178            -8.683700          -3.706722   
4            -9.692178            -8.800099          -3.706722   
..                 ...                  ...                ...   
91           -9.692178            -9.071859          -3.706722   
92           -9.692178            -9.417628          -3.706722   
93           -9.692178            -9.245718          -3.706722   
94           -9.692178            -9.190990          -3.706722   
95           -9.692178           -10.787515          -3.706722   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                  -4.288423e-07   
1            -4.230762                  -4.288423e-07   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.063654           -10.099939          -3.886203   
1           -10.063654           -10.602896          -3.886203   
2           -10.063654            -9.569157          -3.886203   
3           -10.063654            -8.683700          -3.886203   
4           -10.063654            -8.800099          -3.886203   
..                 ...                  ...                ...   
91          -10.063654            -9.071859          -3.886203   
92          -10.063654            -9.417628          -3.886203   
93          -10.063654            -9.245718          -3.886203   
94          -10.063654            -9.190990          -3.886203   
95          -10.063654           -10.787515          -3.886203   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000013   
1            -4.230762                      -0.000013   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.66936           -10.099939          -3.704429   
1             -9.66936           -10.602896          -3.704429   
2             -9.66936            -9.569157          -3.704429   
3             -9.66936            -8.683700          -3.704429   
4             -9.66936            -8.800099          -3.704429   
..                 ...                  ...                ...   
91            -9.66936            -9.071859          -3.704429   
92            -9.66936            -9.417628          -3.704429   
93            -9.66936            -9.245718          -3.704429   
94            -9.66936            -9.190990          -3.704429   
95            -9.66936           -10.787515          -3.704429   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                        0.00001   
1            -4.230762                        0.00001   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: T

In [44]:
print(((accuracy_list.count("No"))/25)*100)
print(mean(range_list))

48.0
309.4166666666667


# APPROACH 4 (10 most similar cells)

## VARIANCE

In [45]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the most similar cells
    most_similar_cells = df.nsmallest(10, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])
    number6 = int(y_train[most_similar_cells.iloc[5].name])
    number7 = int(y_train[most_similar_cells.iloc[6].name])
    number8 = int(y_train[most_similar_cells.iloc[7].name])
    number9 = int(y_train[most_similar_cells.iloc[8].name])
    number10 = int(y_train[most_similar_cells.iloc[9].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5, number6, number7, number8, number9, number10]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

    cell_test  cell_train  distance
0   -10.21142  -10.099939  0.111482
1   -10.21142  -10.602896  0.391476
2   -10.21142   -9.569157  0.642263
3   -10.21142   -8.683700  1.527721
4   -10.21142   -8.800099  1.411322
..        ...         ...       ...
91  -10.21142   -9.071859  1.139561
92  -10.21142   -9.417628  0.793792
93  -10.21142   -9.245718  0.965702
94  -10.21142   -9.190990  1.020430
95  -10.21142  -10.787515  0.576095

[96 rows x 3 columns]
Predicted lifetime range is from:  1009 to  1836
Actual lifetime:  1315


    cell_test  cell_train  distance
0   -9.754075  -10.099939  0.345864
1   -9.754075  -10.602896  0.848821
2   -9.754075   -9.569157  0.184918
3   -9.754075   -8.683700  1.070375
4   -9.754075   -8.800099  0.953976
..        ...         ...       ...
91  -9.754075   -9.071859  0.682216
92  -9.754075   -9.417628  0.336446
93  -9.754075   -9.245718  0.508357
94  -9.754075   -9.190990  0.563085
95  -9.754075  -10.787515  1.033441

[96 rows x 3 columns]
Predicted lifeti

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

In [46]:
print(((accuracy_list.count("No"))/25)*100)
print(mean(range_list))

24.0
521.375


## DISCHARGE

In [51]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(10, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])
    number6 = int(y_train[most_similar_cells.iloc[5].name])
    number7 = int(y_train[most_similar_cells.iloc[6].name])
    number8 = int(y_train[most_similar_cells.iloc[7].name])
    number9 = int(y_train[most_similar_cells.iloc[8].name])
    number10 = int(y_train[most_similar_cells.iloc[9].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5, number6, number7, number8, number9, number10]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -10.21142           -10.099939          -3.995719   
1            -10.21142           -10.602896          -3.995719   
2            -10.21142            -9.569157          -3.995719   
3            -10.21142            -8.683700          -3.995719   
4            -10.21142            -8.800099          -3.995719   
..                 ...                  ...                ...   
91           -10.21142            -9.071859          -3.995719   
92           -10.21142            -9.417628          -3.995719   
93           -10.21142            -9.245718          -3.995719   
94           -10.21142            -9.190990          -3.995719   
95           -10.21142           -10.787515          -3.995719   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -3.319977            -0.741646   
1            -4.230762           -3.319977            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Predicted lifetime range is from:  713 to  1115
Actual lifetime:  989


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.692178           -10.099939          -3.706722   
1            -9.692178           -10.602896          -3.706722   
2            -9.692178            -9.569157          -3.706722   
3            -9.692178            -8.683700          -3.706722   
4            -9.692178            -8.800099          -3.706722   
..                 ...                  ...                ...   
91           -9.692178            -9.071859          -3.706722   
92           -9.692178            -9.417628          -3.706722   
93           -9.692178            -9.245718          -3.706722   
94           -9.692178            -9.190990          -3.706722   
95           -9.692178           -10.787515          -3.706722   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -1.167855            -0.741646   


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipyker

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.063654           -10.099939          -3.886203   
1           -10.063654           -10.602896          -3.886203   
2           -10.063654            -9.569157          -3.886203   
3           -10.063654            -8.683700          -3.886203   
4           -10.063654            -8.800099          -3.886203   
..                 ...                  ...                ...   
91          -10.063654            -9.071859          -3.886203   
92          -10.063654            -9.417628          -3.886203   
93          -10.063654            -9.245718          -3.886203   
94          -10.063654            -9.190990          -3.886203   
95          -10.063654           -10.787515          -3.886203   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -3.922042           -0.957516            -0.741646   
1            -4.230762           -0.957516            -1.052154   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [52]:
print(((accuracy_list.count("No"))/25)*100)
print(mean(range_list))

16.0
551.0833333333334


## FULL MODEL

In [49]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(10, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])
    number6 = int(y_train[most_similar_cells.iloc[5].name])
    number7 = int(y_train[most_similar_cells.iloc[6].name])
    number8 = int(y_train[most_similar_cells.iloc[7].name])
    number9 = int(y_train[most_similar_cells.iloc[8].name])
    number10 = int(y_train[most_similar_cells.iloc[9].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5, number6, number7, number8, number9, number10]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -10.21142           -10.099939          -3.995719   
1            -10.21142           -10.602896          -3.995719   
2            -10.21142            -9.569157          -3.995719   
3            -10.21142            -8.683700          -3.995719   
4            -10.21142            -8.800099          -3.995719   
..                 ...                  ...                ...   
91           -10.21142            -9.071859          -3.995719   
92           -10.21142            -9.417628          -3.995719   
93           -10.21142            -9.245718          -3.995719   
94           -10.21142            -9.190990          -3.995719   
95           -10.21142           -10.787515          -3.995719   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000006   
1            -4.230762                      -0.000006   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.560392           -10.099939          -3.656142   
1            -9.560392           -10.602896          -3.656142   
2            -9.560392            -9.569157          -3.656142   
3            -9.560392            -8.683700          -3.656142   
4            -9.560392            -8.800099          -3.656142   
..                 ...                  ...                ...   
91           -9.560392            -9.071859          -3.656142   
92           -9.560392            -9.417628          -3.656142   
93           -9.560392            -9.245718          -3.656142   
94           -9.560392            -9.190990          -3.656142   
95           -9.560392           -10.787515          -3.656142   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                       -0.00003   
1            -4.230762                       -0.00003   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

...

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.88592           -10.099939          -3.826262   
1             -9.88592           -10.602896          -3.826262   
2             -9.88592            -9.569157          -3.826262   
3             -9.88592            -8.683700          -3.826262   
4             -9.88592            -8.800099          -3.826262   
..                 ...                  ...                ...   
91            -9.88592            -9.071859          -3.826262   
92            -9.88592            -9.417628          -3.826262   
93            -9.88592            -9.245718          -3.826262   
94            -9.88592            -9.190990          -3.826262   
95            -9.88592           -10.787515          -3.826262   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                      -0.000021   
1            -4.230762                      -0.000021   
2            -3.646456              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Predicted lifetime range is from:  713 to  1115
Actual lifetime:  786


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.51677           -10.099939          -3.648796   
1             -9.51677           -10.602896          -3.648796   
2             -9.51677            -9.569157          -3.648796   
3             -9.51677            -8.683700          -3.648796   
4             -9.51677            -8.800099          -3.648796   
..                 ...                  ...                ...   
91            -9.51677            -9.071859          -3.648796   
92            -9.51677            -9.417628          -3.648796   
93            -9.51677            -9.245718          -3.648796   
94            -9.51677            -9.190990          -3.648796   
95            -9.51677           -10.787515          -3.648796   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -3.922042                       0.000005   
1            -4.2307

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [50]:
print(((accuracy_list.count("No"))/25)*100)
print(mean(range_list))

28.000000000000004
448.6666666666667
